# Solution 2

### Using xlm-roberta for vectorization and then a simple neural net for label classification

#### Let's load the data.

In [34]:
from pathlib import Path
wd = Path.cwd()
wd = wd.parent.parent
wd = wd / 'merged_data' 
sub1 = str(wd) + '/subtask1.parquet'
print(sub1)

/home/matijak/Documents/programiranje/projects/semeval/merged_data/subtask1.parquet


In [35]:
import pandas as pd
df = pd.read_parquet(sub1)
df.head()

,lang,art_name,entity,start,end,class1,classes2,text
0,BG,BG_670.txt,Запад,152,156,Antagonist,"[Conspirator, Instigator, Foreign Adversary]",Опитът на колективния Запад да „обезкърви Руси...
1,BG,BG_670.txt,САЩ,530,532,Antagonist,[Instigator],Опитът на колективния Запад да „обезкърви Руси...
2,BG,BG_670.txt,НАТО,535,538,Antagonist,[Instigator],Опитът на колективния Запад да „обезкърви Руси...
3,BG,BG_670.txt,Украйна,578,584,Antagonist,[Foreign Adversary],Опитът на колективния Запад да „обезкърви Руси...
4,BG,BG_670.txt,украински войници,633,649,Innocent,[Victim],Опитът на колективния Запад да „обезкърви Руси...


#### Now lets clean article text

In [36]:
import re
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text
df['label'] = df.apply(labelNum,axis=1)
df['input'] = df.apply(cleanText,axis=1)
df.loc[448]

lang                                                       EN
art_name                                     EN_UA_103861.txt
entity                                                Chinese
start                                                     791
end                                                       797
class1                                             Antagonist
classes2                                                [Spy]
text        The World Needs Peacemaker Trump Again \n\n by...
label                                                       0
input       The World Needs Peacemaker Trump Again  by Jef...
Name: 448, dtype: object

In [37]:
def find_all_substring_start_end(text, substring):
    # Use re.finditer to find all occurrences of the substring in the text
    matches = re.finditer(re.escape(substring), text)
    
    # Collect the start and end indices of all matches
    positions = [(match.start(), match.end()) for match in matches]
    
    return positions
def adjust_start_end(row):
    org_text,cl_text,start,end,entity = str(row['text']),str(row['input']),int(row['start']),int(row['end']),str(row['entity'])
    ss1 = find_all_substring_start_end(org_text,entity)
    ss2 = find_all_substring_start_end(cl_text,entity)
    #print(ss1,ss2)
    #print(row['text'][start:end])
    a = 0
    for i in range(len(ss1)):
        if abs((ss1[i][0] - start) + (ss1[i][1] - end) ) <= 2:
            a = i
            break
    if org_text[ss1[a][0]:ss1[a][1]] != cl_text[ss2[a][0]:ss2[a][1]]:
        print("ERROR!")
    return ss2[a][0],ss2[a][1]
print(df.loc[0])
df['new_start_end'] = df.apply(adjust_start_end,axis=1)

lang                                                       BG
art_name                                           BG_670.txt
entity                                                  Запад
start                                                     152
end                                                       156
class1                                             Antagonist
classes2         [Conspirator, Instigator, Foreign Adversary]
text        Опитът на колективния Запад да „обезкърви Руси...
label                                                       0
input       Опитът на колективния Запад да „обезкърви Руси...
Name: 0, dtype: object


In [38]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).to(device)
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

def preprocess_function(examples):
    return tokenizer(examples['input'], padding=True, truncation=True,max_length=8192,return_offsets_mapping=True)

data = df.loc[ : , ['input','label','new_start_end','entity']]
data['tokenized']=data.apply(preprocess_function,axis=1)

Using device: cuda


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
print(data.iloc[0]['tokenized'])

{'input_ids': [0, 1089, 22617, 1669, 29, 47829, 2097, 32275, 69, 137, 197, 35359, 53335, 2827, 40053, 155, 135, 128601, 29, 12747, 226, 49, 94511, 137, 2687, 591, 7533, 135, 10099, 54293, 35, 25977, 245, 131732, 155, 35, 18777, 183, 159814, 153, 1089, 22617, 1669, 29, 47829, 2097, 32275, 69, 137, 197, 35359, 53335, 2827, 40053, 155, 135, 128601, 29, 12747, 226, 49, 94511, 137, 2687, 591, 7533, 135, 10099, 54293, 35, 25977, 245, 131732, 155, 35, 18777, 183, 159814, 4629, 69, 62086, 16846, 33318, 4, 3756, 77, 63084, 15258, 1669, 29, 92173, 59, 6208, 29, 6047, 39540, 197, 14114, 16641, 44267, 5, 61216, 193342, 43219, 84535, 2262, 36690, 45961, 213358, 222, 31458, 2549, 29, 45775, 59, 29, 103285, 245, 34078, 29, 40108, 47239, 303, 3512, 105, 22192, 4, 12434, 47853, 19737, 245, 6, 163308, 183, 109560, 205, 29, 40108, 135694, 25223, 650, 447, 3873, 8458, 63522, 5, 44, 123209, 24724, 2374, 205, 29, 40108, 4, 20292, 35, 4907, 155386, 74300, 4301, 61, 51192, 205, 49, 159814, 19173, 40053, 218, 

In [40]:
#print(data.loc[0]['tokenized'])
def indexes(row):
    off_mask = row['tokenized']['offset_mapping']
    start,end = row['new_start_end'][0],row['new_start_end'][1]
    inds = list()
    for p in range(len(off_mask)):
        if off_mask[p][0] >= start and off_mask[p][1] <= end:
            if p != len(off_mask)-1:
                inds.append(p)
    #if len(inds) > 1:
        #print("GREATER THAN 1")
    if len(inds) == 0:
        print(start,end)
    return inds
data['indexes'] = data.apply(indexes,axis=1)

In [41]:
print(data.loc[448],data.loc[1000])

input            The World Needs Peacemaker Trump Again  by Jef...
label                                                            0
new_start_end                                           (785, 792)
entity                                                     Chinese
tokenized              [input_ids, attention_mask, offset_mapping]
indexes                                                      [180]
Name: 448, dtype: object input            जयपुर में जलवायु परिवर्तन को लेकर स्टेट लेवल ट...
label                                                            2
new_start_end                                           (623, 636)
entity                                               केन्द्र सरकार
tokenized              [input_ids, attention_mask, offset_mapping]
indexes                                                 [162, 163]
Name: 1000, dtype: object


In [42]:
data['list'] = data['tokenized'].apply(lambda x: x['input_ids'])
data['attention'] = data['tokenized'].apply(lambda x: x['attention_mask'])
ids = data['list']
att = data['attention']
indexes = data['indexes']
tids = list()
tatt = list()
print(len(ids),len(att),len(indexes))
for i in range(len(ids)):
    tids.append(torch.tensor(ids[i]).to(device))
    tatt.append(torch.tensor(att[i]).to(device))
print(tids[0],tatt[0])
print(indexes[448])

2535 2535 2535
tensor([     0,   1089,  22617,   1669,     29,  47829,   2097,  32275,     69,
           137,    197,  35359,  53335,   2827,  40053,    155,    135, 128601,
            29,  12747,    226,     49,  94511,    137,   2687,    591,   7533,
           135,  10099,  54293,     35,  25977,    245, 131732,    155,     35,
         18777,    183, 159814,    153,   1089,  22617,   1669,     29,  47829,
          2097,  32275,     69,    137,    197,  35359,  53335,   2827,  40053,
           155,    135, 128601,     29,  12747,    226,     49,  94511,    137,
          2687,    591,   7533,    135,  10099,  54293,     35,  25977,    245,
        131732,    155,     35,  18777,    183, 159814,   4629,     69,  62086,
         16846,  33318,      4,   3756,     77,  63084,  15258,   1669,     29,
         92173,     59,   6208,     29,   6047,  39540,    197,  14114,  16641,
         44267,      5,  61216, 193342,  43219,  84535,   2262,  36690,  45961,
        213358,    222,  

In [46]:
sliced_ids = list()
sliced_ntids = list()
sliced_att = list()
key_inds = list()
key_ids = list()

def slices(index,size,context_size):
    if (size<context_size):
        return 0,size
    lower_c = int(context_size/2-1)
    upper_c = int(context_size/2)
    #print(lower_c,upper_c)
    if index < lower_c:
        return 0,context_size
    elif index >= lower_c:
        if index + upper_c > size:
            return index-(context_size-(size-index)), size
        else:
            return index-lower_c,index+upper_c+1  


for i in range(len(tids)):
    slower,supper = slices(indexes[i][0],len(tids[i]),40)
    key_id = ids[i][indexes[i][0]]
    #key_tid = tids[i][indexes[i][0]]
    pid = ids[i][slower:supper]
    key_inds.append([])
    for j in indexes[i]:
        
        key_id = ids[i][j]
        if key_id not in pid:
           print(len(ids[i]),key_id,slower,supper,indexes[i])
        key_inds[i].append(pid.index(key_id))
    sliced_ids.append(tids[i][slower:supper])
    sliced_att.append(tatt[i][slower:supper])

Min = 10000
for i in range(len(indexes)):
    if len(indexes[i]) < Min:
        Min = len(indexes[i])
#print(Min)
#print(len(sliced_ids[448]),len(tids[448]),sliced_ids[0],sliced_att[0])
print(key_inds[448],tids[448][99])

[19] tensor(76438, device='cuda:0')


In [47]:
from torch.utils.data import DataLoader, TensorDataset

input_ids_batch = torch.nn.utils.rnn.pad_sequence(sliced_ids, batch_first=True, padding_value=0)
attention_mask_batch = torch.nn.utils.rnn.pad_sequence(sliced_att, batch_first=True, padding_value=0)

dataset = TensorDataset(input_ids_batch, attention_mask_batch)

dataloader = DataLoader(dataset, batch_size=100)

ind = 0

vectors = []

#print(input_ids_batch)
for batch in dataloader:
    #print(len(batch[0]),batch)
    input_for_model = {
        "input_ids": batch[0],
        "attention_mask" : batch[1]
    }

    with torch.no_grad():  # Disable gradients for inference
        outputs = model(**input_for_model,output_hidden_states=True)
    hidden_states = outputs.hidden_states
    last_hs = hidden_states[-1]
    #print(last_hs.shape)
    for i in range(len(last_hs)):
        vectors.append([])
        for j in range(len(key_inds[ind])):
            vectors[len(vectors)-1].append(last_hs[i][key_inds[ind][j]])
        ind+=1        

In [48]:
print(vectors[500])
#print(vectors[448],key_inds[448])

[tensor([ 6.0182e-02,  9.6063e-02, -2.4367e-03,  2.3522e-04,  1.4157e-01,
        -1.9084e-01, -8.9138e-02,  2.4629e-01,  8.1022e-02,  3.3121e-02,
        -1.5948e-02,  4.0185e-02,  2.4637e-01, -6.7611e-02, -2.0017e-02,
         7.8462e-02,  1.9462e-02, -2.9817e-02,  1.0935e-01, -6.4263e-02,
        -1.3107e-01,  1.9458e-02, -5.3467e-03,  5.3320e-02,  8.4158e-02,
        -6.5116e-02,  1.8467e-02, -4.7020e-02,  1.1844e-01, -9.7403e-02,
         8.4644e-02,  4.4193e-02, -7.0033e-02, -2.3572e-02,  4.5031e-02,
         7.6761e-02,  8.1940e-02, -3.7150e-02,  1.3652e-01, -1.2604e-02,
         1.1577e-01,  4.1190e-02, -1.9032e-03,  6.6562e-02,  1.9405e-02,
        -2.0590e-02, -3.8596e-02,  1.9137e-02,  6.8997e-02, -3.5650e-02,
        -4.4676e-04,  8.4557e-02,  1.6037e-02,  6.6177e-02,  1.5282e-02,
        -9.7177e-02,  1.1209e-01, -1.0169e-01,  2.8294e-02,  1.5552e-01,
        -5.2742e-02,  2.1420e-02, -2.7690e-02, -1.7846e-01,  1.4243e-01,
        -9.2559e-04, -1.1287e-01,  4.3927e-04, -7.

In [49]:
X = list()
for i in range(len(vectors)):
    Sum = 0
    for j in range(len(vectors[i])):
        Sum += vectors[i][j]
    X.append(Sum)
print(len(X))
print(type(X))
X = torch.stack(X)
print(type(X))

2535
<class 'list'>
<class 'torch.Tensor'>


In [50]:
y = data['label']
y = torch.tensor(y).to(device)
print(y[1000],X[1000])

tensor(2, device='cuda:0') tensor([-3.0607e-02, -4.8388e-03,  4.3151e-02,  1.0151e-01,  1.2123e-01,
        -1.1919e-01, -4.7051e-02,  4.7089e-01,  8.6444e-02, -8.4271e-02,
         3.7279e-01,  1.5769e-02,  1.1586e+00,  1.2930e-01,  1.2015e-01,
        -8.8587e-02,  4.6082e-02, -8.1174e-02,  5.1833e-03,  1.6612e-01,
         2.4407e-02,  4.7630e-02,  5.4761e-02,  1.3848e-01,  7.4771e-02,
        -5.1961e-02,  9.2482e-02, -4.3879e-02, -1.6148e-01,  2.1498e-01,
         1.4433e-02,  4.4248e-02, -2.7221e-02, -5.1553e-01,  2.9440e-02,
         5.4720e-03, -1.9554e-01,  1.0129e-02,  5.7673e-02, -6.7010e-02,
         1.0623e-01,  2.7086e-01,  7.2336e-02,  2.3089e-01,  1.2077e-01,
        -1.6994e-02,  5.9846e-02,  1.8232e-01, -1.8017e-02, -1.1366e-01,
         6.7394e-02,  4.5340e-01, -5.4690e-02,  5.1189e-01,  2.8533e-01,
        -1.6626e-02, -7.3314e-02,  1.3215e-01,  3.2630e-02, -1.6430e-01,
         7.6323e-02,  7.4527e-02, -1.0001e-01, -4.6767e-01,  1.8547e-01,
        -2.2468e-02, -2.

In [72]:
from sklearn.model_selection import train_test_split
#print(len(X))
#print(X.shape)  # Should print: torch.Size([length_of_sequence])
#print(type(X))#X_Stack = torch.stack(X)
#print(X_Stack.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=12
)
print(X[0])
#print(X_train.shape,X_test.shape,)

tensor([ 6.9675e-02,  7.7524e-02, -4.8405e-02,  8.7762e-02, -2.1373e-02,
        -7.0729e-02, -1.3044e-02,  2.2310e-01,  3.5856e-02, -3.9805e-02,
        -2.8858e-02,  9.9903e-02,  1.1901e-01,  7.0323e-02,  7.1627e-03,
         8.1264e-02,  2.9464e-02,  2.1906e-02,  3.3575e-02,  1.5326e-02,
         1.0041e-01,  2.4231e-02,  2.8257e-02,  1.0127e-01,  9.3648e-02,
        -1.2036e-01,  4.5067e-02, -3.8167e-02,  1.5165e-01, -4.0828e-02,
         5.3078e-02, -8.6856e-02, -8.1753e-02, -9.0155e-03,  8.3507e-02,
         6.0432e-02,  8.2966e-04, -2.4601e-02, -2.2407e-01,  7.2635e-02,
         1.3449e-01, -1.0850e-01,  2.9732e-03,  2.9165e-02,  2.9882e-02,
         5.0211e-02,  1.6391e-02,  1.0101e-04, -3.7670e-03, -7.1267e-02,
         4.0323e-02, -1.0894e-01,  9.6292e-02,  8.9507e-04, -6.3622e-02,
        -4.7164e-02,  8.2625e-02,  1.0176e-02, -1.8386e-01,  1.2664e-01,
         7.4938e-02,  1.5429e-02,  7.2286e-02, -6.2298e-02, -6.0989e-02,
         1.7736e-02, -5.1333e-02, -4.2081e-03, -1.5

In [80]:
from torch.optim import AdamW
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test,y_test)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64)
optimizer = AdamW(model.parameters(), lr=2e-5)

In [81]:
import torch.nn as nn
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        # Input to hidden layer
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        # Second hidden layer
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        # Output layer
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

model2 = SimpleNN(input_size=768, hidden_size=128, output_size=3)
model2.to(device)

SimpleNN(
  (fc1): Linear(in_features=768, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=3, bias=True)
)

In [82]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

num_epochs = 50
criterion = nn.CrossEntropyLoss()
for epoch in range(num_epochs):
    # Training phase
    model2.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    # Initialize tqdm progress bar for training
    train_progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}")
    
    for batch in train_progress_bar:
        optimizer.zero_grad()
        
        # Move data and labels to device (CPU or GPU)
        data, labels = batch
        data = data.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model2(data)  # Shape will be (batch_size, 3)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate accuracy
        _, preds = torch.max(outputs, 1)  # Correct way to get predicted class
        #print(preds)
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)
        
        # Update tqdm description with current loss
        train_progress_bar.set_postfix({'loss': loss.item()})
    
    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = correct_predictions / total_predictions
    
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")
    
    # Test phase
    model2.eval()
    test_loss = 0
    correct_test_predictions = 0
    total_test_predictions = 0
    
    test_progress_bar = tqdm(test_dataloader, desc=f"Test Epoch {epoch + 1}/{num_epochs}")
    
    with torch.no_grad():
        for batch in test_progress_bar:
            data, labels = batch
            data = data.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model2(data)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            # Calculate test accuracy
            preds = torch.argmax(outputs, dim=-1)  # Correct way to get predicted class
            #print(preds)
            correct_test_predictions += (preds == labels).sum().item()
            total_test_predictions += labels.size(0)
            
            test_progress_bar.set_postfix({'loss': loss.item()})
    
    avg_test_loss = test_loss / len(test_dataloader)
    test_accuracy = correct_test_predictions / total_test_predictions
    
    print(f"Test loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")


Training Epoch 1/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 352.80it/s, loss=1.11]


Epoch 1/50
Training loss: 1.0947, Training accuracy: 0.4638


Test Epoch 1/50: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 423.38it/s, loss=1.11]


Test loss: 1.0984, Test accuracy: 0.4409


Training Epoch 2/50: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 404.70it/s, loss=1.1]


...

Test loss: 1.0984, Test accuracy: 0.4409


Training Epoch 50/50: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 410.55it/s, loss=1.1]


Epoch 50/50
Training loss: 1.0946, Training accuracy: 0.4638


Test Epoch 50/50: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 571.31it/s, loss=1.11]

Test loss: 1.0984, Test accuracy: 0.4409
